# Testing tabulate to output tables

In [1]:
!pip install tabulate

In [ ]:
# from tabulate import tabulate
# data = [ ["Earth",6371,888885973.600001],
#          ["Moon",1737,73.5],
#          ["Mars",3390,641.85]]
# tabulate(data, tablefmt='html', headers=['distance'], colalign=("center",))

# Import packages to parse results

In [9]:
from glob import glob
import pandas as pd
import json
#import numpy as np
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# import plotly.io as pio
# pio.renderers.default = 'iframe'
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# Used for debug
# from pprint import pprint

## Log file locations relative to notebook

### Create a dataframe with all json data

In [36]:
def create_dataframe(logs):
    logs = glob(logs)
    df_list = []
    for log in logs:
        with open(log) as f:
            obj = json.load(f)['saaf']
        # cpuInfo is messing up the dataframe... remove for now
        obj.pop('cpuInfo')
        df_list.append(pd.DataFrame([obj]))
    return pd.concat(df_list, ignore_index=True)

### For SEBS, we have 3 different clusters, 4 methods, multiple concurrent calls

In [45]:
# kOps clusters (running on AWS)
# AWS EKS cluster
clusters={
    'kops_8x_8vcpu': '../logs/openfaas/kops_multi-tenant_spot_1x_72vcpu_144gb/sebs',
    'kops_1x_72vcpu': '../logs/openfaas/kops_multi-tenant_spot_8x_8vcpu_16gb/sebs',
    'eks_8x_8vcpu': '../logs/openfaas/eks_multi-tenant_spot_8x_8vcpu_16gb/sebs'
}
methods=('dna', 'bfs', 'mst', 'pagerank')
concurrent_calls=(32, 40, 48, 56, 64)

sebs_df=dict()
for k, v in clusters.items():
    sebs_df[k]=dict()
    for m in methods:
        sebs_df[k][m]=dict()
        for c in concurrent_calls:
            sebs_df[k][m][c]=create_dataframe(f"{v}/concurrent-{c}/*{m}*")